# ICAPS 2024 Tutorial: Finding multiple plans - SymK

In order to use SymK, it is necessary to install it together with the Unified Planning Library where SymK is integrated.

**Note**: If you want to use the latest version of SymK with the best raw performance and newest features, it might be worth to check the [github repository of SymK](https://github.com/speckdavid/symk) and install it as a standalone tool.

In [31]:
from IPython.display import clear_output

# SymK (integrated in the unified-planning library)
!pip install unified-planning==1.1.0
!pip install up-symk==1.3.1
clear_output(wait=False)

In [11]:
import sys
from collections import defaultdict

import unified_planning as up
import up_symk
from unified_planning.shortcuts import *
from unified_planning.io import PDDLReader

## Domain and Problem Files
We use the iconic problem number one of the gripper domain, where the goal is to move two
balls from room A to room B with a gripper that has two arms.

We need to download the domain and problem files described in PDDL for the planning task we aim to solve. Following this, we read and use these files to create a PDDL problem, setting the quality metric to minimize the plan length.

In [3]:
!wget https://raw.githubusercontent.com/speckdavid/up-symk/master/notebooks/gripper-domain.pddl
!wget https://raw.githubusercontent.com/speckdavid/up-symk/master/notebooks/gripper-prob01.pddl
clear_output(wait=False)

In [13]:
reader = PDDLReader()
pddl_problem = reader.parse_problem("gripper-domain.pddl", "gripper-prob-two-balls.pddl")
if len(pddl_problem.quality_metrics) == 0:
    pddl_problem.add_quality_metric(MinimizeSequentialPlanLength())

## Validator
We define two functions that serve as helper functions to evaluate the found plans and calculate the cost of the plans.

In [14]:
def get_plan_cost(problem, plan):
    pv = PlanValidator(problem_kind=problem.kind)
    pv_res = pv.validate(problem, plan)
    return pv_res.metric_evaluations[problem.quality_metrics[0]]

def get_plans_by_cost(problem, plans):
    pv = PlanValidator(problem_kind=problem.kind)
    plans_by_cost = defaultdict(lambda: [])
    for plan in plans:
        pv_res = pv.validate(problem, plan)
        cost = pv_res.metric_evaluations[problem.quality_metrics[0]]
        plans_by_cost[cost].append(result.plan)
    return plans_by_cost

## Finding a single optimal solution
We solve our gripper problem optimally with ``symk-opt``.

In [15]:
with OneshotPlanner(name='symk-opt') as planner:
    result = planner.solve(pddl_problem) # output_stream=sys.stdout
    if result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
        cost = get_plan_cost(pddl_problem, result.plan)
        print(f"{planner.name} found this plan: {result.plan} with cost {cost}.")
    else:
        print("No plan found.")
        
# We do not want to see the credits again and again, so let's disable them from now on.
up.shortcuts.get_environment().credits_stream = None

SymK (with optimality guarantee) found this plan: SequentialPlan:
    pick(ball2, rooma, right)
    pick(ball1, rooma, left)
    move(rooma, roomb)
    drop(ball1, roomb, left)
    drop(ball2, roomb, right) with cost 5.


## Finding multiple optimal solutions
We solve our gripper problem with ``symk-opt`` and ask for three optimal solutions, which are reported.

In [16]:
plans = []

with AnytimePlanner(name='symk-opt', params={"number_of_plans": 3}) as planner:
    for i, result in enumerate(planner.get_solutions(pddl_problem)): # output_stream=sys.stdout: 
        if result.status == up.engines.PlanGenerationResultStatus.INTERMEDIATE:
            plans.append(result.plan)
            print(f"Plan {i+1}: {result.plan} with cost {get_plan_cost(pddl_problem, result.plan)}.\n")
        elif result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
            for cost, plans in get_plans_by_cost(pddl_problem, plans).items():
                print(f"{planner.name} found {len(plans)} optimal plans with cost {cost}.")
        elif result.status not in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print("No plan found.") 

Plan 1: SequentialPlan:
    pick(ball2, rooma, right)
    pick(ball1, rooma, left)
    move(rooma, roomb)
    drop(ball1, roomb, left)
    drop(ball2, roomb, right) with cost 5.

Plan 2: SequentialPlan:
    pick(ball2, rooma, right)
    pick(ball1, rooma, left)
    move(rooma, roomb)
    drop(ball2, roomb, right)
    drop(ball1, roomb, left) with cost 5.

Plan 3: SequentialPlan:
    pick(ball1, rooma, left)
    pick(ball2, rooma, right)
    move(rooma, roomb)
    drop(ball1, roomb, left)
    drop(ball2, roomb, right) with cost 5.

SymK (with optimality guarantee) found 3 optimal plans with cost 5.


## Finding all optimal solutions
We find all optimal solutions (using ``symk-opt``) and report the number of existing optimal plans and their cost. Note that by default the number of plans generated in Anytime mode is 1, so you need to specify the number (None sets it to infinite).

In [29]:
plans = []

with AnytimePlanner(name='symk-opt', problem_kind=pddl_problem.kind, anytime_guarantee="OPTIMAL_PLANS", 
                    params={"number_of_plans": None}) as planner:
    for i, result in enumerate(planner.get_solutions(pddl_problem)): # output_stream=sys.stdout): 
        if result.status == up.engines.PlanGenerationResultStatus.INTERMEDIATE:
            plans.append(result.plan)
            if i > 0 and i % 10 == 0:
                print(f"{planner.name} found {i} plans...")
        elif result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print(f"\n{planner.name} found {i} plans!\n")
        elif result.status not in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print("No plan found.")
            
# Calculate the cost of the plans found
print("Calculate the cost of the plans found...")
for cost, plans in get_plans_by_cost(pddl_problem, plans).items():
    print(f"{planner.name} found {len(plans)} plans with cost {cost}.")


SymK (with optimality guarantee) found 8 plans!

Calculate the cost of the plans found...
SymK (with optimality guarantee) found 8 plans with cost 5.


## Finding multiple solutions ordered by cost
Again, we solve our gripper problem and this time we ask for 50 plans, of which 8 plans are optimal.

In [30]:
# Find 500 plans with SymK
plans = []

with AnytimePlanner(name='symk', params={"number_of_plans": 50}) as planner:
    for i, result in enumerate(planner.get_solutions(pddl_problem)): # output_stream=sys.stdout): 
        if result.status == up.engines.PlanGenerationResultStatus.INTERMEDIATE:
            plans.append(result.plan)
            if i > 0 and i % 10 == 0:
                print(f"{planner.name} found {i} plans...")
        elif result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print(f"\n{planner.name} found {i} plans!\n")
        elif result.status not in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print("No plan found.")

# Calculate the cost of the plans found
print("Calculate the cost of the plans found...")
for cost, plans in get_plans_by_cost(pddl_problem, plans).items():
    print(f"{planner.name} found {len(plans)} plans with cost {cost}.")

SymK found 10 plans...
SymK found 20 plans...
SymK found 30 plans...
SymK found 40 plans...

SymK found 50 plans!

Calculate the cost of the plans found...
SymK found 8 plans with cost 5.
SymK found 8 plans with cost 6.
SymK found 34 plans with cost 7.


# Finding Loopless Plans

Finally, you can also request to find only loopless plans by adding it to the param section.

In [25]:
# Find 500 plans with SymK
plans = []

with AnytimePlanner(name='symk', params={"number_of_plans": 50, "loopless": True}) as planner:
    for i, result in enumerate(planner.get_solutions(pddl_problem)): # output_stream=sys.stdout): 
        if result.status == up.engines.PlanGenerationResultStatus.INTERMEDIATE:
            plans.append(result.plan)
            if i > 0 and i % 100 == 0:
                print(f"{planner.name} found {i} plans...")
        elif result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print(f"\n{planner.name} found {i} plans!\n")
        elif result.status not in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print("No plan found.")

# Calculate the cost of the plans found
print("Calculate the cost of the plans found...")
for cost, plans in get_plans_by_cost(pddl_problem, plans).items():
    print(f"{planner.name} found {len(plans)} plans with cost {cost}.")


SymK found 50 plans!

Calculate the cost of the plans found...
SymK found 8 plans with cost 5.
SymK found 8 plans with cost 6.
SymK found 8 plans with cost 7.
SymK found 8 plans with cost 8.
SymK found 18 plans with cost 9.
